Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the related LOs requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>CS152 Assignment 2: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [2]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

class Literal:
    def __init__(self, name, sign=True):
        self.name = name
        self.sign = sign
    def __neg__(self):
        return(Literal(self.name, not self.sign))
    def __eq__(self, other):
        return(self.name == other.name and self.sign == other.sign)
    def __str__(self):
        if self.sign:
            return(self.name)
        else:
            return("-" + self.name)
    def __hash__(self):
        return(hash(str(self)))    


<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [75]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below
import copy

# FOR DEBUGGING
def printkb(kb):
    print(">>>")
    for k in kb:
        print([str(c) for c in k])
    print("<<<")
    return(0)


def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    ss = {d.name for v in KB for d in v} # set of unassigned symbols
    
    def DPLL(cs, ss, m, s, o):
        
        '''
        cs = clauses
        ss = symbols
        m = model
        '''
        # >>> DEBUG
        print("###")
        print('try',s)
        print(ss)
        printkb(cs)
        print("###")
        # DEBUG <<<
        
        # prevent reference error
        ucs = copy.deepcopy(cs)
        nss = copy.deepcopy(ss)
        
        # remove clauses that have s
        ucs = [c for c in ucs if s not in c]
        # remove not s from clauses
        [c.discard(-s) for c in ucs]
        
        # update model
        if s.sign: m[s] = True
        else: m[-s] = False
        
        # update ordered list of assignment
        o.append(s)
        
        # >>> DEBUG
        print("check")
        printkb(ucs)
        # DEBUG <<<
        
        if not ucs:
            return(True, m, s)
        elif set() in ucs:
            return(False, {}, s)
        else:
            s = nss.pop()
            s = Literal(s)
            print('branching')
            resP = DPLL(ucs,nss,m,s,o)
            if resP: return(resP)
            else: return(DPLL(ucs,nss,m,-s,o))
    s = Literal(ss.pop())
    print('hola',ss)
    print(DPLL(KB,ss,{},s, []))
    print('hola',ss)
    print(DPLL(KB,ss,{},-s,[]))
    return(0)

In [76]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A,B,E},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]

DPLL_Satisfiable(KB,0)

hola {'B', 'E', 'C', 'D', 'A'}
###
try F
{'B', 'E', 'C', 'D', 'A'}
>>>
['-A', 'B', 'E']
['-E', 'D']
['-B', '-C', '-F']
['-E', 'B']
['-B', 'F']
['-B', 'C']
<<<
###
check
>>>
['-A', 'B', 'E']
['-E', 'D']
['-B', '-C']
['-E', 'B']
['-B', 'C']
<<<
branching
###
try B
{'E', 'C', 'D', 'A'}
>>>
['-A', 'B', 'E']
['-E', 'D']
['-B', '-C']
['-E', 'B']
['-B', 'C']
<<<
###
check
>>>
['-E', 'D']
['-C']
['C']
<<<
branching
###
try D
{'C', 'A', 'E'}
>>>
['-E', 'D']
['-C']
['C']
<<<
###
check
>>>
['-C']
['C']
<<<
branching
###
try C
{'A', 'E'}
>>>
['-C']
['C']
<<<
###
check
>>>
[]
<<<
(False, {}, <__main__.Literal object at 0x1122fd8d0>)
hola {'B', 'E', 'C', 'D', 'A'}
###
try -F
{'B', 'E', 'C', 'D', 'A'}
>>>
['-A', 'B', 'E']
['-E', 'D']
['-B', '-C', '-F']
['-E', 'B']
['-B', 'F']
['-B', 'C']
<<<
###
check
>>>
['-A', 'B', 'E']
['-E', 'D']
['-E', 'B']
['-B']
['-B', 'C']
<<<
branching
###
try B
{'E', 'C', 'D', 'A'}
>>>
['-A', 'B', 'E']
['-E', 'D']
['-E', 'B']
['-B']
['-B', 'C']
<<<
###
check
>>>
['-E', 'D']

0

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [5]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A,B,E},{-E,D},{-C,-F,-B},{-E,B},{-B,F},{-B,C}]

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [6]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [7]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [9]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

symbols to be processes {'A', 'B'}
processing C
before
['C', 'A']
['C', '-A']
['-C', 'B']
______
after
[{<__main__.Literal object at 0x10bd79310>}]
['B']
______
False
symbols to be processes {'B'}
processing A
before
['B']
______
after
[{<__main__.Literal object at 0x10bd79310>}]
['B']
______
False
symbols to be processes set()
processing B
before
['B']
______
after
[]
______
False
symbols to be processes {'A'}
processing C
before
['C', 'A']
['C', '-A']
['-C']
______
after
[set()]
[]
______
True


In [10]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

symbols to be processes {'F', 'D', 'B', 'E', 'A'}
processing C
before
['E', '-A', 'B']
['D', '-E']
['-C', '-B', '-F']
['B', '-E']
['F', '-B']
['C', '-B']
______
after
[{<__main__.Literal object at 0x10bd79410>, <__main__.Literal object at 0x10bd79490>, <__main__.Literal object at 0x10bd79310>}, {<__main__.Literal object at 0x10bd793d0>, <__main__.Literal object at 0x10bd79510>}, {<__main__.Literal object at 0x10bd795d0>, <__main__.Literal object at 0x10bd79590>}, {<__main__.Literal object at 0x10bd79310>, <__main__.Literal object at 0x10bd79610>}, {<__main__.Literal object at 0x10bd79450>, <__main__.Literal object at 0x10bd79650>}]
['E', '-A', 'B']
['D', '-E']
['-B', '-F']
['B', '-E']
['F', '-B']
______
False
symbols to be processes {'D', 'B', 'E', 'A'}
processing F
before
['E', '-A', 'B']
['D', '-E']
['-B', '-F']
['B', '-E']
['F', '-B']
______
after
[{<__main__.Literal object at 0x10bd79410>, <__main__.Literal object at 0x10bd79490>, <__main__.Literal object at 0x10bd79310>}, {<__main

AssertionError: 

<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_list</code> needs to be correctly generated 

In [ ]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['B','F', 'E', 'C', 'C', 'E', 'A'])

In [ ]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'])

In [ ]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] == ['D', 'B', 'A', 'C', 'F', 'E', 'A'])